# Introduction

This notebook will pull a Spotify playlist's data using Spotipy libraries and Spotify API.

For more information, please visit https://spotipy.readthedocs.io/en/latest/. Watch https://www.youtube.com/watch?v=vipVEWe86Lg&t=242s for more information

The required inputs for this notebooks are as followings: 
- Spotify Client ID and Secret key, which can be found in Spotify Dashboard
- User's Spotify ID
- Playlist

Note: Spotify API limits the number of tracks pulled up to 49 tracks. Therefore, the notebook contains a loop to automate this process

Import Libraries

In [ ]:
import urllib.request
from urllib.request import urlopen

import sys
import spotipy
import spotipy.util as util
import os
import json
import webbrowser
from json.decoder import JSONDecodeError

import random

Expand Screen

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Input

In [ ]:
playlist_id = input('Enter playlist ID: ')
playlist_id = playlist_id[17:]

In [ ]:
USERNAME = sys.argv[1] 
CLIENT_ID = client_id# Go to Spotify App Developer Dashboard
CLIENT_SECRET = client_secret # Go to Spotify App Developer Dashboard
REDIRECT_URI = 'http://google.com/' # Set it as 'http://google.com/'
SCOPE = scope # will give user to modify

# Body Script

###### Get access using API

In [ ]:
def run_spotify_api():
    try:
        token = util.prompt_for_user_token(username = USERNAME, 
                                       scope = SCOPE, 
                                       client_id = CLIENT_ID, 
                                       client_secret = CLIENT_SECRET, 
                                       redirect_uri = REDIRECT_URI)
    except:
        os.remove(f".cache~{username}")
        token = util.prompt_for_user_token(username = USERNAME, 
                                       scope = SCOPE, 
                                       client_id = CLIENT_ID, 
                                       client_secret = CLIENT_SECRET, 
                                       redirect_uri = REDIRECT_URI)
    sp = spotipy.Spotify(auth=token)
    return sp

### Get User's Playlist

In [ ]:
# Function to show track
def show_tracks(tracks):
    track_list = []
    for i, item in enumerate(tracks['items']):
        track = item['track']
        track_list.append((i, track['artists'][0]['name'], track['name'], track['album']['name']))
    return track_list

In [ ]:
def run_spotify_get_playlist_tracks(playlist_id):
    # Get playlist from Spotify API
    playlist = sp.user_playlist(user_spotify_id, playlist_id)
    print(playlist['name'])
    print('Total tracks', playlist['tracks']['total'])

    tracks = playlist['tracks']
    my_playlist = show_tracks(tracks)
    while tracks['next']:
        tracks = sp.next(tracks)
        my_playlist = show_tracks(tracks)
    
    # Convert from tuple to list
    for i in range(len(my_playlist)):
        my_playlist[i] = list(my_playlist[i])

    # Remove character ' in the track data
    for i in range(len(my_playlist)):
        for y in range(1,3):
            my_playlist[i][y] = my_playlist[i][y].replace("'","")
    return my_playlist

### Get Track's ID

In [ ]:
def search_track_id(track_name, artist, album):
    # Create a dictionary for result
    search_result = dict.fromkeys(['track_name', 'artist', 'album', 'track_id', 'popularity'])
    
    # Limit number of raw result output of 10 and market in US
    # Search by Track name first
    if album == '':
        album = '{}'
    else: 
        album = album
       
    searched_raw_result = sp.search(q="album: "+ album + " artist: " + artist + " track: " + track_name, limit=1, type='track', market=None)
    # Search by Artist name
    searched_raw_result1 = searched_raw_result['tracks']['items']
    
    if len(searched_raw_result1) >0:
        search_result.update(track_name = searched_raw_result1[0]['name'],
                             artist = searched_raw_result1[0]['album']['artists'][0]['name'], 
                             album = searched_raw_result1[0]['album']['name'],
                             track_id = searched_raw_result1[0]['uri'][14:],
                             popularity = searched_raw_result1[0]['popularity'])
    return search_result

### Get Track Data from User's Playlist

In [ ]:
def get_data(tracks_selection, my_playlist):
    playlist_data = [] 

    for i in tracks_selection:
        # Get Track ID by track name, artist and album collected from the playlist
        track_find_id = search_track_id(track_name = my_playlist[i][2], artist = my_playlist[i][1], album = my_playlist[i][3])
        if track_find_id['track_id'] is not None:
        # Get Track Audio Features
            track_audio_features = sp.audio_features(track_find_id['track_id'])

            # Get Track Audio Analysis
            audio_analysis_dict = dict.fromkeys(['sections'])
            audio_analysis_dict.update(sections = sp.audio_analysis(track_find_id['track_id'])['sections'])

            # Merge 2 dictionaries into one
            track_audio_data = dict(track_audio_features[0], **audio_analysis_dict)
            track_data = dict(track_find_id, **track_audio_data)

            playlist_data.append(track_data)
    return playlist_data

### Run Notebook and Export File

In [ ]:
sp = run_spotify_api()
my_playlist = run_spotify_get_playlist_tracks(playlist_id)

# Spotify API allows only 100 tracks per playlist. Later we can combine data
playlist_data = get_data(range(len(my_playlist)), my_playlist)

In [ ]:
playlist_name = sp.user_playlist(user_spotify_id, playlist_id)['name']
print(playlist_name)

In [ ]:
import json
json = json.dumps(playlist_data)

f = open(local_address +  user_spotify_name + '_' + playlist_name + "_playlist_data.json","w")
f.write(json)
f.close()